<a href="https://colab.research.google.com/github/ROARMarketingConcepts/Recurrent-Neural-Network-Examples/blob/master/Static_RNN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
import tensorflow as tf

In [0]:
n_inputs = 3
n_neurons = 5

In [0]:
reset_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1],dtype=tf.float32)
Y0, Y1 = output_seqs

In [0]:
init = tf.global_variables_initializer()

In [0]:
import numpy as np

# Mini-batch:       instance 0, instance 1, instance 2, instance 3

X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

In [0]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [0]:
Y0_val # (4,5) tensors for each of the 4 instances of X0

array([[ 0.30741334, -0.32884315, -0.65428466, -0.9385059 ,  0.52089024],
       [ 0.99122757, -0.95425403, -0.75180787, -0.9995208 ,  0.9820235 ],
       [ 0.9999268 , -0.99783254, -0.8247353 , -0.9999963 ,  0.9994779 ],
       [ 0.996771  , -0.68750614,  0.8419969 ,  0.93039113,  0.81206834]],
      dtype=float32)

In [0]:
Y1_val # (4,5) tensors for each of the 4 instances of X1 and Y0

array([[ 0.999989  , -0.99976057, -0.0667929 , -0.9999803 ,  0.99982214],
       [-0.6524943 , -0.51520866, -0.37968946, -0.59225935, -0.08968391],
       [ 0.99862397, -0.99715203, -0.03308637, -0.9991566 ,  0.9932902 ],
       [ 0.99681675, -0.9598194 ,  0.3966063 , -0.83076054,  0.7967197 ]],
      dtype=float32)

### Packing Sequences

In [0]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))      # make n_steps (time steps) first dimension. Unstack tensors for each time step.

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs,dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [0]:
init = tf.global_variables_initializer()

In [0]:
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [0]:
print(outputs_val) # tensors of dimension (2,3)*(3,5) = (2,5) for each instance

[[[-0.45652324 -0.68064123  0.40938237  0.63104504 -0.45732826]
  [-0.94288    -0.999887    0.9405581   0.9999984  -0.9999998 ]]

 [[-0.80015343 -0.9921827   0.78177965  0.9971032  -0.9964609 ]
  [-0.637116    0.11300927  0.5798437   0.4310559  -0.6371699 ]]

 [[-0.93605185 -0.9998379   0.9308867   0.9999814  -0.9999831 ]
  [-0.9165386  -0.9945605   0.896054    0.99987197 -0.9999751 ]]

 [[ 0.9927367  -0.9981933  -0.55543643  0.9989031  -0.9953323 ]
  [-0.02746338 -0.7319198   0.7827872   0.9525681  -0.978177  ]]]


In [0]:
print(np.transpose(outputs_val, axes=[1, 0, 2])[1])

[[-0.94288    -0.999887    0.9405581   0.9999984  -0.9999998 ]
 [-0.637116    0.11300927  0.5798437   0.4310559  -0.6371699 ]
 [-0.9165386  -0.9945605   0.896054    0.99987197 -0.9999751 ]
 [-0.02746338 -0.7319198   0.7827872   0.9525681  -0.978177  ]]
